# Машинный перевод и озвучивание видеозаписей на русском языке

## О проекте

[Есть](https://www.youtube.com/watch?v=p3lsYlod5OU&ab_channel=LexFridman) интересная беседа [Лекса Фридмана](https://en.wikipedia.org/wiki/Lex_Fridman) с [Михаилом Левиным](https://en.wikipedia.org/wiki/Michael_Levin_(biologist)). Несмотря, на русское происхождение обоих собеседников разговор они ведут на английском. Цель проекта перевести и озвучить беседу на русском языке.

В проекте используется [готовая расшифровка с таймингом](https://karpathy.ai/lexicap/0325-large.html), созданная с помощью пакета [OpenAI Wisper](https://github.com/openai/whisper).
Далее описывается парсинг расшифровки, перевод на русский язык, машинное озвучивание, выравнивание длины русских фрагментов с соответствующими оригинальными.

Данный подход можно использовать для перевода и озвучивания любой видеозаписи.

## Устанавливаем библиотеки

In [ ]:
# Библиотека для работы c Google Translate
%pip install googletrans==3.1.0a0

# Библиотека для генерации tts
%pip install pyttsx3

# Прогресс-бар
%pip install tqdm

## Импортируем необходимые для работы модули

In [ ]:
from tqdm import tqdm
from bs4 import BeautifulSoup
from datetime import datetime
from scipy.io.wavfile import read, write
from googletrans import Translator

import os
import pyttsx3
import requests
import numpy as np
import pandas as pd


## Парсинг расшифровки с таймингом

Для парсинга используем библиотеку `BeatifulSoup`:

In [ ]:
url = 'https://karpathy.ai/lexicap/0325-large.html'
data = requests.get(url).text
soup = BeautifulSoup(data)

## Обработка английского текста

Выделим фрагменты текста и их тайминги в отдельные списки:

In [ ]:
t_divs = soup.find_all('div', {'class': 't'})
en_text = []
for div in t_divs:
    en_text.append(div.text)
    # if len(div.text) < 66:
    #     text += '\n'

s_divs = soup.find_all('div', {'class': 's'})
timing = []
for div in s_divs:
    timing.append(div.a.text)

Для удовлетворительного качества необходимо переводить текст как минимум целыми предложениями. В качестве знаков для разделения используем точку и вопросительный знак.

In [ ]:
def sign_finder(sign, str):
    '''Finds the last position of the sing in the string'''
    rev_pos = str[::-1].find(sign)
    if rev_pos == -1:
        return 0    
    pos = len(str) - rev_pos
    return pos
def pos_finder(text):
    signs = ['.', '?']
    pos = []
    for sign in signs:
        pos.append(sign_finder(sign, text))
    return max(pos)

Перенесём незаконченные фрагменты предложений к фрагментам, содержащим знаки конца предложения, и внесём соответствующие поправки в тайминг.

In [ ]:
dense_text = []
new_timing = []
buffer = []
time = []
for i, text in enumerate(en_text):
    pos = pos_finder(text)
    if (pos == 0) & (i < len(en_text) - 1):
        buffer.append(text)
        time.append(timing[i])
    elif len(buffer) > 0:
        for i in range(len(buffer)):
            text = buffer[-1] + text
        dense_text.append(text)
        new_timing.append(time[0])
        buffer.clear()
        time.clear()
    else:
        dense_text.append(text)
        new_timing.append(timing[i])


Из данных списков создадим таблицу `Pandas`: 

In [ ]:
dict = {'timing': new_timing, 'en_text': dense_text}
df = pd.DataFrame(dict)
df

Тайминги переведем в формат `DateFrame` и вычислим продожительность оригинальных фрагментов, сохранив их в отдельный столбец:

In [ ]:
format = '%H:%M:%S.%f'
def duration_calculator(row):
    '''Calculates duration speech fragment in a row'''
    idx = row.name
    time1 = row['timing'] + '000'
    if idx == len(df) - 1:
        return datetime.strptime("03:00:20.000000", format) - datetime.strptime(time1, format)    
    time2 = df.loc[idx + 1, 'timing'] + '000'
    return datetime.strptime(time2, format) - datetime.strptime(time1, format)

df['duration'] = df.apply(duration_calculator, axis=1)
df['duration'] = df['duration'].apply(lambda x: x.total_seconds())
df

Посмотрим на длину фрагментов:

In [ ]:
df.en_text.apply(len).describe()

Самый короткий 5 символов, а длинный -- 574. 

Для перевода фрагментов текста в отдельные предложения (группы предложений) и определения соответствующей им временной продолжительности мне будет удобнее работать со списками, а не таблицей.

In [ ]:
text = list(df.en_text)
duration = list(df.duration)

Будем ориентироваться на знаки окончания предложения в конце фрагмента:

In [ ]:
def min_pos_finder(text):
    '''Returns the first position of the signs in the text'''
    signs = ['.', '?']
    pos = []
    for sign in signs:
        pos.append(text.find(sign))
    if min(pos) == -1:
        return max(pos) + 1
    else:
        return min(pos) + 1

Собственно переносим, что нужно в тексте и вносим, соответствующие изменения в их длительность:

In [ ]:
for i, row in enumerate (text):
    if i != 0:
        pos = min_pos_finder(row)
        head = row[:pos]
        tail = row[pos:]
        duration[i - 1] = duration[i - 1] + duration[i] * len(head) / len(row)
        duration[i] = duration[i] * len(tail) / len(row)
        text[i - 1] = text[i - 1] + head
        text[i] = tail         


In [ ]:
size = 0
for row in text:
    size += len(row)

print('Количество знаков в исходном тексте: ', df.en_text.apply(len).sum())
print('Количество знаков итоговом тексте: ', size)
print('Исходная общая продолжительность: ', sum(duration))
print('Общая продолжительность после обработки: ', df.duration.sum())


Всё на месте. Возваращаем данные обратно в таблицу.

In [ ]:
df['en_text'] = text
df['duration'] = duration
df

## Перевод текста на русский

Переводим текст на русский:

In [ ]:
translator = Translator()
ru_text = []
for row in tqdm(text):
    ru_text.append(translator.translate(row, src='en', dest='ru').text)

Короткими фрагментами переводить данный текст всё же долго. Кроме того, вероятно переводчик может учитывать контекст, что положительным образом должно сказываться на переводе. Однако наш текстовый блок слишком большой для одномоментного перевода, переводчик не справиться. Желательно подобрать максимальный размер группы текстов опытным путём, у меня это 90. Меньше - дольше, а больше - переводчик не справляется. Однако необходимо вставить подходящий маркер для обратного разъединения строк с чем мне не удалось справиться. Но код, на всякий случай, оставляю.

In [ ]:
# print(len(df))
# translator = Translator()
# ru_text = []

# batch_size = 80
# start = 0
# for i in tqdm(range(round(len(df) / batch_size))):    
#     batch = df.en_text[start:start+batch_size]
    
#     en_batch = '*'.join(batch)
#     ru_batch = translator.translate(en_batch, src='en', dest='ru').text
#     # print(1, ru_batch[:100])
#     ru_text.extend(ru_batch.split('*'))
#     start += batch_size
# len(ru_text)

In [ ]:
for i, row in enumerate(ru_text):
    if row == '':
        ru_text.pop(i)
len(ru_text)

In [ ]:
df['ru_text'] = ru_text

Посмотрим что получилось:

In [ ]:
for row in ru_text[:10]:
    print(row)

Боже мой! "размером примерно два сантиметра в сантиметре или два в размере".

С некоторыми фрагментами текста неплохо справляется переводчик `DeepL`, но, к сожалению, перевод текстов более 5 000  знаков требует подписки, а для россиян, в данный момент, она не предусмотрена.

Изменения в текст перевода можно внести, например, так:

In [ ]:
ru_text[0] = 'Оказывается, если обучить планарий, а затем отрезать им голову, хвост регенерирует совершенно новый мозг, который все еще помнит первоначальную информацию. Я думаю, что в планариях кроется ответ практически на все глубокие вопросы жизни.'
ru_text[2] = 'У них много различных внутренних органов, но они такие маленькие, они размером примерно два сантиметра на один.'

Создадим заново датафрейм с текстами на двух языках, их продолжительностью, и сохраним его, при необходимости, в `csv`-файл.

In [ ]:
df = pd.DataFrame({'duration': duration, 'en_text': text, 'ru_text': ru_text})
df.to_csv('levin.csv', index=False)

## Машинное озвучивание текста и подготовка к монтажу

Теперь, даже после перерыва, мы можем загрузить таблицу из файла:

In [ ]:
df = pd.read_csv('levin.csv')

In [ ]:
for i in range(10):
    print(i, df.ru_text[i])

Я использую голос, который загрузил с сайта https://rhvoice.su/

In [ ]:
tts = pyttsx3.init()

voices = tts.getProperty('voices')
  
for voice in voices:
    # to get the info. about various voices in our PC 
    print("Voice:")
    print("ID: %s" %voice.id)
    print("Name: %s" %voice.name)
    print("Age: %s" %voice.age)
    print("Gender: %s" %voice.gender)
    print("Languages Known: %s" %voice.languages)

In [ ]:
# Задать голос по умолчанию

tts.setProperty('voice', 'ru') 
tts.setProperty('rate', 300)
tts.setProperty('volume', .8)

# Устанавливаем нужный голос
RU_VOICE_ID = "HKEY_LOCAL_MACHINE\SOFTWARE\Microsoft\Speech\Voices\TokenEnums\RHVoice\Aleksandr-hq"
# RU_VOICE_ID = 'HKEY_LOCAL_MACHINE\SOFTWARE\Microsoft\Speech\Voices\Tokens\TTS_MS_RU-RU_IRINA_11.0'


tts.setProperty('voice', RU_VOICE_ID)

Проверим настройки:

In [ ]:
speech = 'Привет! Давайте проверим скорость и качество озвучивания'
tts.say(speech)
tts.runAndWait()

Конечно, качество не очень. Можно попробовать использовать Yandex.SpeachKit или использовать модели машинного обучения.

Создадим набор звуковых файлов из текстовых фрагментов:

In [ ]:
os.mkdir('files')
for i in tqdm(range(len(df))):
    speech = df.ru_text[i]
    file_name = 'files/' + f'{i}.wav'
    tts.save_to_file(speech, file_name)
    tts.runAndWait()


## Обработка звука

Сравним длину полученных фрагментов с длительностью оригинальных фрагментов:

In [ ]:
durs = []
for i in range(len(df)):
    file_name = 'files/' + f'{i}.wav'
    fs, data = read(file_name)
    durs.append(round(len(data) / fs, 3))   
sum(durs)

Общая длительность озвучивания 8106 секунд,  меньше общей длительности видеозаписи -- 10820 секунд.

Найдём разность между оригинальной длительностью фрагмента и русской озвучкой:

In [ ]:
df['ru_duration'] = durs
df['delta_duration'] = df['duration'] - df['ru_duration']

In [ ]:
df

In [ ]:
df.delta_duration.describe()

Оригинальный фрагмент может быть на 20 секунд длиннее, а может и на 3 секунды короче.

In [ ]:
df.delta_duration.sum() + df.ru_duration.sum()

Общая продолжительность совпадает с оригинальной.

Если длительности озвученных фрагментов превышает оригинал, то заберём это время у следующих фрагментов, обладающих запасом по времени. Работать будем со списком. 

In [ ]:
deltas = list(df.delta_duration)

In [ ]:
time = 0
for i, delta in enumerate(deltas):
    
    if delta < 0:        
        time += delta
        deltas[i] = 0
    elif time < 0:
        if abs(time) <= delta:
            deltas[i] += time
            time = 0
        else:
            time += delta 
            deltas[i] = 0       

In [ ]:
print(df.duration.sum())
print(df.ru_duration.sum() + df.delta_duration.sum())

Общее время не поменялось.

Добавим информацию в таблицу.

In [ ]:
df['delta_duration'] = deltas

In [ ]:
df

Добавим после каждого вновь озвученного звукового фрагмента, продолжительностью меньше оригинального, заполненной тишиной спейсер, необходимой продолжительности:

In [ ]:
fs = 22050
for i, delta in enumerate(deltas):
    if delta > 0:
        file_name = f'files/{i}-1.wav'
        multiplicator = int(delta * fs)
        spacer = np.array([0] * multiplicator)
        write(file_name, fs, spacer)

Можно попробовать "сшить" фрагменты русской озвучки со спейсерами с помощью утилиты [`ffmpeg`](https://ffmpeg.org/download.html). 

In [ ]:
with open("list.txt", "w") as f:
    for i in range(len(df)):
        filename = f'files/{i}.wav'
        line = f"file '{filename}'\n"
        if deltas[i] > 0:
            line += f"file 'files/{i}-1.wav'\n"
        f.write(line)

In [ ]:
os.system("ffmpeg -f concat -i out/list.txt -c copy out/output.wav")

По неизвестной мне причине, итоговый файл у меня получился размером больше, чем должен был. Я использовал для объединения утилиту плеера `AIMP`, указав папку `files` как источник и один файл с раширением `.mp3`. Последний вариант хорошо сработал и я получил на выходе звуковой файл продолжительностью 3 часа 20 секунд (10820 секунд).

Далее я загружаю оригинальный файл с видео на ПК с помощью сервиса [SaveFrom](https://ru.savefrom.net/1-%D0%B1%D1%8B%D1%81%D1%82%D1%80%D1%8B%D0%B9-%D1%81%D0%BF%D0%BE%D1%81%D0%BE%D0%B1-%D1%81%D0%BA%D0%B0%D1%87%D0%B0%D1%82%D1%8C-%D1%81-youtube-130/?url=http%3A%2F%2Fyoutube.com%2Fwatch%3Fv%3Dp3lsYlod5OU&ab_channel=LexFridman&utm_source=youtube.com&utm_medium=short_domains&utm_campaign=ssyoutube.com&a_ts=1666604287.019) и в видеоредакторе приглушаю оригинальную видеодорожку и добавляю вновь созданную. Можно попробовать обойтись средствами утилиты `ffmpeg`.


## Общий вывод

Результат можно посмотреть [здесь](https://youtu.be/nJh880lnlRA)

На мой взгляд результат получился неплохой. Однако качество перевода удовлетворительное, но далеко не идеальное, в этом направлении можно поработать. Также следует улучшить качество озвучивания. Можно попробовать реализовать на основе данного проекта суфлёр и озвучить текст перевода самостоятельно.